# How to run reaction procedures
This is a streamlined approach to running reaction procedures, distilling the essence of our advanced method. While our full methodology leverages multiprocessor software integrated with a PostGRES database, this simplified framework operates on the same fundamental principles, providing a more accessible and easy-to-understand overview of the process.

In [1]:
import json
from pathlib import Path
from itertools import product
from typing import List
import pandas as pd
from rdkit import Chem
from rdkit.Chem import rdChemReactions
from models import (
    Substructure, 
    ReactantsToFind, 
    Reaction, 
    ReactionProcedure, 
    Homopolymer, 
    Molecule, 
    ReactionStep, 
    PolymerizationReaction
)

In [2]:
with open('reaction_procedure.json') as f:
    rps = json.load(f)

In [3]:
folder = Path('polymers')

dfs = []
for i in range(20):
    fileid = str(i)
    if i < 10:
        fileid = '0' + fileid
    dfs.append(pd.read_csv(folder / f'split_{fileid}.csv'))

df = pd.concat(dfs, ignore_index=True)

In [4]:
molecules = df.monomer_smiles.to_list()[0:1000]

In [5]:
# run first reaction procedure
rp = ReactionProcedure(**rps[0]['reaction_procedure'])

In [6]:
def find_reactants_per_step(reactions, molecules):
    """Finds the possible reactants per each step of the reaction

    The first key indicates which reaction step the reactant is used in, and
    the second key indicates which location in the reactants list the reactant
    will be used in. The first key could be variable (for example, the key could be
    0, 1, 2 if there are three steps, or it could be just 1 or just 0). The second one
    should always be sequential (e.g., 1, 2, 3) if three reactants possible for a step.
    """
    reactants_per_step = {}
    for step, reaction in enumerate(reactions):
        reactants_to_find = reaction.reactants_to_find
        if reactants_to_find is None or len(reactants_to_find) == 0:
            continue
        reactants_per_step[step] = {}
        for i in range(len(reactants_to_find)):
            reactants_per_step[step][i] = []
        for mol in molecules:
            invalid = False
            m = Chem.MolFromSmiles(mol)
            for i, reactant in enumerate(reactants_to_find):
                for substructure in reactant.unacceptable_substructures:
                    patt = Chem.MolFromSmarts(substructure.smarts)
                    if m is None or m.HasSubstructMatch(patt):
                        invalid = True
                        break
                if invalid:
                    continue
                for substructure in reactant.necessary_substructures:
                    patt = Chem.MolFromSmarts(substructure.smarts)
                    if m is not None and m.HasSubstructMatch(patt):
                        count = len(m.GetSubstructMatches(patt))
                        if count < substructure.greater_than_or_equal or count > substructure.less_than_or_equal:
                            invalid = True
                            break
                    else:
                        invalid = True
                        break
                if not invalid:
                    reactants_per_step[step][i].append(mol)
    return reactants_per_step

In [7]:
def build_molecule_pairings(reactants_per_step):
    """Enumerates over possible pairings of all reactants"""
    step_pairings = {}
    for step, reactants in reactants_per_step.items():
        step_pairings[step] = [x for x in product(*[v for v in reactants.values()])]
    pairings = [dict(zip(step_pairings.keys(), p)) for p in product(*step_pairings.values())]
    return pairings

In [8]:
def build_reactions(rp, pairings):
    """Build reaction objects"""
    reactions = []
    for pairing in pairings:
        pr = PolymerizationReaction(monomer_generation_step=rp.monomer_generation_step, reactions=[])
        for i, reaction in enumerate(rp.reactions):
            step = ReactionStep(additional_reactants=reaction.additional_reactants, smarts=reaction.smarts)
            if i in pairing:
                step.molecules = [Molecule(smiles=x) for x in pairing[i]]
            pr.reactions.append(step.copy(deep=True))
        reactions.append(pr.copy(deep=True))
    return reactions

In [9]:
def polymerize(pr: PolymerizationReaction) -> Homopolymer:
    """Polymerizes molecules using passed in polymerization reaction and
    returns homopolymer or None if fails
    """
    rxns, step_reactants = [], []
    for step in pr.reactions:
        rxns.append(rdChemReactions.ReactionFromSmarts(step.smarts))
        reactants = [Chem.MolFromSmiles(mol.smiles) for mol in step.molecules]
        reactants.extend([Chem.MolFromSmiles(mol) for mol in step.additional_reactants])
        step_reactants.append(reactants)
    monomer = None
    if pr.monomer_generation_step is not None and pr.monomer_generation_step == 0:
        if len(pr.reactions[0].molecules) == 1:
            monomer = pr.reactions[0].molecules[0].smiles
        else:
            print("Can't determine which molecule is the monomer")
    try:
        reactants = ()
        for i, rxn in enumerate(rxns):
            reactants += tuple(step_reactants[i])
            product = rxn.RunReactants(reactants)[0][0]
            Chem.SanitizeMol(product)
            product.UpdatePropertyCache()
            reactants = (product,)
            if (
                pr.monomer_generation_step is not None
                and i + 1 == pr.monomer_generation_step
            ):
                monomer = Chem.MolToSmiles(reactants[0])

        smiles = Chem.MolToSmiles(reactants[0])
        smiles = smiles.replace("[*", "*").replace("*]", "*").replace("*", "[*]")
        hp = Homopolymer(
            polymer_smiles=smiles,
            monomer_smiles=monomer,
        )
        return hp
    except Exception as e:
        print(f"Error polymerizing {pr}")
        print(e)
        return None

In [10]:
reactants_per_step = find_reactants_per_step(rp.reactions, molecules)
pairings = build_molecule_pairings(reactants_per_step)
reactions = build_reactions(rp, pairings)
polymers = [polymerize(reaction) for reaction in reactions]

In [11]:
polymers = pd.DataFrame([dict(x) for x in polymers])

In [12]:
polymers

,polymer_smiles,monomer_smiles
0,[*]SCCC(CCNC(=O)C1(N)CC1)(CC([*])(C)C)c1ccccc1,CC1(C)CC(CCNC(=O)C2(N)CC2)(c2ccccc2)CCS1
1,[*]CCC1C(NC(=O)c2csc(-c3ccccc3)n2)C2(CCC2)C1S[*],O=C(NC1C2CCSC2C12CCC2)c1csc(-c2ccccc2)n1
2,[*]CCC[C@H](CS[*])NC(=O)[C@]1(c2ccccc2)C[C@H](...,O=C(N[C@@H]1CCCSC1)[C@]1(c2ccccc2)C[C@H](O)C1
3,[*]CC[C@H]1C(Nc2ccc3c(cnn3C(F)F)c2)C2(CCC2)[C@...,FC(F)n1ncc2cc(NC3[C@@H]4CCS[C@@H]4C34CCC4)ccc21
4,[*]CC[C@H]1C(Nc2cccc(-n3cccn3)c2)C2(CCC2)[C@H]...,c1cc(NC2[C@@H]3CCS[C@@H]3C23CCC3)cc(-n2cccn2)c1
...,...,...
315,[*]SCCC1(C2(CN)CC(O)C2)CCC[C@H]1[*],NCC1(C23CCC[C@H]2SCC3)CC(O)C1
316,[*]CCC(CS[*])N1CC(C(=O)Nc2ccc(Nc3ccccc3)cc2)C1,O=C(Nc1ccc(Nc2ccccc2)cc1)C1CN(C2CCSC2)C1
317,[*]CCC1C(NCc2ncn(-c3ccccc3)n2)C2(CCC2)C1S[*],c1ccc(-n2cnc(CNC3C4CCSC4C34CCC4)n2)cc1
318,[*]CC[C@H]1C(NCc2ncn(-c3ccccc3)n2)C2(CCC2)[C@H...,c1ccc(-n2cnc(CNC3[C@@H]4CCS[C@@H]4C34CCC4)n2)cc1
